In [ ]:
# This is my code which compares pictures in original project lmdb and hdfd5
# actually we train test on different pictures which not allow us to directly compare losses

In [1]:
import lmdb
import caffe
import struct
import numpy as np
import hashlib
import h5py
import json

In [102]:
db = lmdb.open("/home/anatolix/iidf-data/Realtime_Pose_Estimation_LMDB", readonly=True )

In [170]:
def process(key, array):
    metadata=array[3]
    
    #0 dataset name
    dataset = struct.unpack('@10s',metadata[0].tobytes()[:10])[0]
    dataset = dataset.partition(b'\x00')[0]
    
    #1 image height width
    height, width = struct.unpack('ff',metadata[1].tobytes()[:8])
    
    #2 validation, numother, people_index, 4*annolist_index, write_count
    validation, numother, people_index, annolist_index, write_count, total_count  = struct.unpack('<?BBfff',metadata[2].tobytes()[:15])

    # repeated for each person
    #3 objpos_x, objpos_y 
    #4 scale_povided
    #4,5,6 joint self 3*16
    
    #print(dataset, height, width, validation, numother, people_index, annolist_index, write_count, total_count)

    idx=3

    objpos = []
    joints = []
    
    objpos_x, objpos_y = struct.unpack('<ff',metadata[idx].tobytes()[:8])
    idx += 1
    objpos.append([objpos_x, objpos_y])
    
    scale_povided = struct.unpack('<f',metadata[idx].tobytes()[:4])[0]
    idx += 1

    #print(objpos_x, objpos_y, scale_povided)

    joints_x = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
    idx += 1        
    joints_y = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
    idx += 1        
    joints_v = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
    idx += 1        
    #print(joints_x, joints_y, joints_v,"\n")
    joints.append([joints_x, joints_y, joints_v])
    
    for i in range(numother):
        objpos_x, objpos_y = struct.unpack('<ff',metadata[idx].tobytes()[:8])
        idx += 1
        #print(objpos_x, objpos_y)
        objpos.append([objpos_x, objpos_y])
        
    scale_povided = struct.unpack('<'+'f'*numother,metadata[idx].tobytes()[:4*numother])
    #print(scale_povided)
    idx += 1        
        
    for i in range(numother):
        joints_x = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
        idx += 1        
        joints_y = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
        idx += 1        
        joints_v = struct.unpack('f'*16,metadata[idx].tobytes()[:64])
        idx += 1        
        #print(joints_x, joints_y, joints_v,"\n")
        joints.append([joints_x, joints_y, joints_v])

    return (dataset.decode('ascii'), validation, int(annolist_index), int(write_count), int(total_count), int(height), int(width), np.array(joints).transpose([0,2,1]).astype(int))


In [193]:
accum = []
with open("hashes_lmdb.txt","w") as f:
    with db.begin() as env:
        cursor = env.cursor()
        i=0
        for key, raw_datum in cursor:
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(raw_datum)
            array = caffe.io.datum_to_array(datum)
            dataset, validation, annolist_index, write_count, total_count, height, width, joints  = process(key, array)
            jhash = hashlib.sha1(joints.tobytes()).hexdigest()
            phash = hashlib.sha1(array[:3,:,:].tobytes()).hexdigest()
            print(i, dataset,validation, annolist_index, write_count, total_count, height, width, jhash, phash, joints.tolist(), file=f)
            i=i+1

    
    

In [ ]:
db.close()

In [2]:
h5 = h5py.File('dataset/val_dataset.h5', "r")
datum = h5['datum']
keys = list(datum.keys())

with open("hashes_val.txt","w") as f:

    for key in keys:
        entry = datum[key]
        meta = json.loads(entry.attrs['meta'])
        img = meta['img_path']
        height = meta['img_height']
        width = meta['img_width']
        isValidation = meta['isValidation']
        dataset = meta['dataset']
        joints = np.array(meta['joints']).astype(int)
        jhash = hashlib.sha1(joints.tobytes()).hexdigest()
        phash = hashlib.sha1(entry.value[:3,:,:]).hexdigest()
       
        print(img, dataset, bool(isValidation), height,width,jhash,phash, joints.tolist(), file=f)



In [3]:
h5 = h5py.File('dataset/train_dataset.h5', "r")
datum = h5['datum']
keys = list(datum.keys())

with open("hashes_train.txt","w") as f:

    for key in keys:
        entry = datum[key]
        meta = json.loads(entry.attrs['meta'])
        img = meta['img_path']
        height = meta['img_height']
        width = meta['img_width']
        isValidation = meta['isValidation']
        dataset = meta['dataset']
        joints = np.array(meta['joints']).astype(int)
        jhash = hashlib.sha1(joints.tobytes()).hexdigest()
        phash = hashlib.sha1(entry.value[:3,:,:]).hexdigest()
        
        print(img, dataset, bool(isValidation), height,width,jhash,phash, joints.tolist(), file=f)

